In [1]:
import gc
import time
import pandas as pd
import numpy as np
import csv
import pywt
import os
from sklearn.externals import joblib
from sklearn.metrics import f1_score,hamming_loss
from sklearn.metrics import precision_recall_fscore_support as prf
import warnings
warnings.filterwarnings("ignore")
path = "/media/jdcloud/"

In [2]:
def preprocess_y(labels, y, num_class=10):
    bin_label = np.zeros((len(y), num_class)).astype('int8')
    for i in range(len(y)):
        label_nona = labels.loc[y[i]].dropna()
        for j in range(1, label_nona.shape[0]):
            bin_label[i, int(label_nona[j])] = 1
    return bin_label

In [10]:
train = []

#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold0.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold1.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_attention_maxpooling_10net_fold2.csv").values)
path = './'

# train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_4block_10net_fold2.csv").values) #1fold f0.822

#train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold0.csv").values)
#train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_one_net_fold2.csv").values) # 1fold f0.813

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold1.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_1net_fold2.csv").values)

train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold0.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_fold2.csv").values) # 3folds f0.817

train.append(pd.read_csv(path+"ensemble_csv/"+"densenet_f0819_10net_fold.csv").values) # 1fold f0.819

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_sym_addori_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_sym_addori_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_sym_addori_fold2.csv").values) # 3folds f0.817

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_db6_addori_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_db6_addori_fold1.csv").values)
train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_db6_addori_fold2.csv").values)

# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_ori_addori_fold0.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_ori_addori_fold1.csv").values)
# train.append(pd.read_csv(path+"ensemble_csv/"+"attention_10net_ori_addori_fold2.csv").values)

In [17]:
# out = np.hstack(test)#
out0 = np.hstack(train[:-1]) #
out1 = np.hstack(train[:-2])
out2 = np.hstack(train[:-2]+[train[-1]])

LR_clf = joblib.load("LR_ensemble0810.pkl")
MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")
MLkNN10_clf = joblib.load("MLkNN10_ensemble.pkl")
MLkNN10_db6_clf = joblib.load("MLkNN10_ensemble_db6_0810.pkl")

y_pred_LR = LR_clf.predict(out0)
y_pred_proba_LR = LR_clf.predict_proba(out0)

y_pred_MLkNN8 = MLkNN8_clf.predict(out1).toarray()
y_pred_proba_MLkNN8 = MLkNN8_clf.predict_proba(out1).toarray()

y_pred_MLkNN10 = MLkNN10_clf.predict(out1).toarray()
y_pred_proba_MLkNN10 = MLkNN10_clf.predict_proba(out1).toarray()

y_pred_MLkNN10_db6 = MLkNN10_db6_clf.predict(out2).toarray()
y_pred_proba_MLkNN10_db6 = MLkNN10_db6_clf.predict_proba(out2).toarray()

y_pred_LR[:,2] = y_pred_MLkNN8[:,2]
y_pred_proba_LR[:,2] = y_pred_proba_MLkNN8[:,2]

y_pred_LR[:,5] = y_pred_MLkNN8[:,5]
y_pred_proba_LR[:,5] = y_pred_proba_MLkNN8[:,5]

y_pred_LR[:,6] = y_pred_MLkNN8[:,6]
y_pred_proba_LR[:,6] = y_pred_proba_MLkNN8[:,6]

y_pred_LR[:,9] = y_pred_MLkNN8[:,9]
y_pred_proba_LR[:,9] = y_pred_proba_MLkNN8[:,9]


y_pred_LR[:,0] = y_pred_MLkNN10[:,0]
y_pred_proba_LR[:,0] = y_pred_proba_MLkNN10[:,0]

y_pred_LR[:,4] = y_pred_MLkNN10[:,4]
y_pred_proba_LR[:,4] = y_pred_proba_MLkNN10[:,4]

y_pred_LR[:,8] = y_pred_MLkNN10[:,8]
y_pred_proba_LR[:,8] = y_pred_proba_MLkNN10[:,8]

y_pred_LR[:,7] = y_pred_MLkNN10_db6[:,7]
y_pred_proba_LR[:,7] = y_pred_proba_MLkNN10_db6[:,7]


y_pred = y_pred_LR
y_pred_proba_train = y_pred_proba_LR

# y_pred = clf.predict(x_train)#.toarray()
# y_pred_proba_train = clf.predict(x_train)#.toarray()


In [18]:
labels = pd.read_csv("/media/uuser/data/final_codes/final_run_test/REFERENCE.csv")

index = np.arange(6689)
y_train = preprocess_y(labels, index)

print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))
for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(y_train, y_pred)) 
print(" train data precision recall f1  :", prf(y_train, y_pred,average="samples"))# 'micro', 'weighted'

 train data f1_score  : 0.9488485550878499
f1 score of ab 0 is 0.9718297798267139
f1 score of ab 1 is 0.9915591932865238
f1 score of ab 2 is 0.9668607152468933
f1 score of ab 3 is 0.9864456171309433
f1 score of ab 4 is 0.9586664545859218
f1 score of ab 5 is 0.9790363885692794
f1 score of ab 6 is 0.9739862764690532
f1 score of ab 7 is 0.9609072469746974
f1 score of ab 8 is 0.9571900723380558
f1 score of ab 9 is 0.9623326213834353
 train data hamming_loss  : 0.011122738824936463
 train data precision recall f1  : (0.9567947376289431, 0.9624831813425028, 0.9556355257506401, None)


In [ ]:
 train data f1_score  : 0.9486110583264719
f1 score of ab 0 is 0.9701732528678535
f1 score of ab 1 is 0.9915591932865238
f1 score of ab 2 is 0.9668607152468933
f1 score of ab 3 is 0.9864456171309433
f1 score of ab 4 is 0.9586664545859218
f1 score of ab 5 is 0.9790363885692794
f1 score of ab 6 is 0.9739862764690532
f1 score of ab 7 is 0.9609072469746974
f1 score of ab 8 is 0.9571900723380558
f1 score of ab 9 is 0.9623326213834353
 train data hamming_loss  : 0.011257288084915534
 train data precision recall f1  : (0.9544775003737479, 0.9612871879204666, 0.9536920364398314, None)


In [ ]:
 train data f1_score  : 0.9481877348787832
f1 score of ab 0 is 0.9718297798267139
f1 score of ab 1 is 0.9904887607485836
f1 score of ab 2 is 0.9668607152468933
f1 score of ab 3 is 0.985171184676515
f1 score of ab 4 is 0.9586664545859218
f1 score of ab 5 is 0.9790363885692794
f1 score of ab 6 is 0.9739862764690532
f1 score of ab 7 is 0.9596956215380122
f1 score of ab 8 is 0.9571900723380558
f1 score of ab 9 is 0.9623326213834353
 train data hamming_loss  : 0.011212438331589176
 train data precision recall f1  : (0.9557233268550356, 0.961137688742712, 0.9544993319997057, None)


In [15]:

LR_clf = joblib.load("LR_ensemble.pkl")
MLkNN_clf = joblib.load("MLkNN10_ensemble.pkl")
MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")

x_train = np.hstack(train[:-2])

y_pred_LR = LR_clf.predict(x_train)
y_pred_MLkNN = MLkNN_clf.predict(x_train).toarray()
y_pred_MLkNN8 = MLkNN8_clf.predict(x_train).toarray()

y_pred_MLkNN[:,1] = y_pred_MLkNN8[:,1]
y_pred_MLkNN[:,2] = y_pred_MLkNN8[:,2]

y_pred_MLkNN[:,3] = y_pred_LR[:,3]

y_pred_MLkNN[:,5] = y_pred_MLkNN8[:,5]
y_pred_MLkNN[:,6] = y_pred_MLkNN8[:,6]

y_pred_MLkNN[:,7] = y_pred_LR[:,7]

y_pred_MLkNN[:,9] = y_pred_MLkNN8[:,9]

y_pred = y_pred_MLkNN

print(" train data f1_score  :", f1_score(y_train, y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(y_train[:, i], y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(y_train, y_pred)) 
print(" train data precision recall f1  :", prf(y_train, y_pred,average="samples"))

 train data f1_score  : 0.9481877348787832
f1 score of ab 0 is 0.9718297798267139
f1 score of ab 1 is 0.9904887607485836
f1 score of ab 2 is 0.9668607152468933
f1 score of ab 3 is 0.985171184676515
f1 score of ab 4 is 0.9586664545859218
f1 score of ab 5 is 0.9790363885692794
f1 score of ab 6 is 0.9739862764690532
f1 score of ab 7 is 0.9596956215380122
f1 score of ab 8 is 0.9571900723380558
f1 score of ab 9 is 0.9623326213834353
 train data hamming_loss  : 0.011212438331589176
 train data precision recall f1  : (0.9557233268550356, 0.961137688742712, 0.9544993319997057, None)


In [3]:
quarter_labels= pd.read_csv("/media/uuser/data/final_codes/final_run_semi/reference.csv")
quarter_index = np.arange(6500)
quarter_y_train = preprocess_y(quarter_labels, quarter_index)

In [4]:
quarter_train = []
path = "./"
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_attention_10net_fold2.csv").values) 

# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold0.csv").values)
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold2.csv").values)

#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold0.csv").values)
#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold0.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_sym_addori_fold2.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_db6_addori_fold2.csv").values)

''' '''
# out = np.hstack(test)#
out0 = np.hstack(quarter_train[:-1])#
out1 = np.hstack(quarter_train[:-2])
out2 = np.hstack(quarter_train[:-2]+[quarter_train[-1]])

LR_clf = joblib.load("LR_ensemble0810.pkl")
MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")
MLkNN10_clf = joblib.load("MLkNN10_ensemble.pkl")
MLkNN10_db6_clf = joblib.load("MLkNN10_ensemble_db6_0810.pkl")

y_pred_LR = LR_clf.predict(out0)
y_pred_proba_LR = LR_clf.predict_proba(out0)

y_pred_MLkNN8 = MLkNN8_clf.predict(out1).toarray()
y_pred_proba_MLkNN8 = MLkNN8_clf.predict_proba(out1).toarray()

y_pred_MLkNN10 = MLkNN10_clf.predict(out1).toarray()
y_pred_proba_MLkNN10 = MLkNN10_clf.predict_proba(out1).toarray()

y_pred_MLkNN10_db6 = MLkNN10_db6_clf.predict(out2).toarray()
y_pred_proba_MLkNN10_db6 = MLkNN10_db6_clf.predict_proba(out2).toarray()

y_pred_LR[:,2] = y_pred_MLkNN8[:,2]
# y_pred_proba_LR[:,2] = y_pred_proba_MLkNN8[:,2]

y_pred_LR[:,5] = y_pred_MLkNN8[:,5]
# y_pred_proba_LR[:,5] = y_pred_proba_MLkNN8[:,5]

y_pred_LR[:,6] = y_pred_MLkNN8[:,6]
# y_pred_proba_LR[:,6] = y_pred_proba_MLkNN8[:,6]

y_pred_LR[:,9] = y_pred_MLkNN8[:,9]
# y_pred_proba_LR[:,9] = y_pred_proba_MLkNN8[:,9]

y_pred_LR[:,0] = y_pred_MLkNN10[:,0]
#y_pred_proba_LR[:,0] = y_pred_proba_MLkNN10[:,0]

y_pred_LR[:,4] = y_pred_MLkNN10[:,4]
# y_pred_proba_LR[:,4] = y_pred_proba_MLkNN10[:,4]

y_pred_LR[:,8] = y_pred_MLkNN10[:,8]
# y_pred_proba_LR[:,8] = y_pred_proba_MLkNN10[:,8]

y_pred_LR[:,7] = y_pred_MLkNN10_db6[:,7]
# y_pred_proba_LR[:,7] = y_pred_proba_MLkNN10_db6[:,7]


quarter_y_pred = y_pred_LR
# quarter_y_pred_proba = y_pred_proba_LR

# quarter_x_train = np.hstack(quarter_train)
# quarter_y_pred = clf.predict(quarter_x_train).toarray()

print(" train data f1_score  :", f1_score(quarter_y_train, quarter_y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(quarter_y_train[:, i], quarter_y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(quarter_y_train, quarter_y_pred)) 
print(" train data precision recall f1  :", prf(quarter_y_train, quarter_y_pred,average="samples"))# 'micro', 'weighted'

 train data f1_score  : 0.8457811834294056
f1 score of ab 0 is 0.9723483730856107
f1 score of ab 1 is 0.988758799327768
f1 score of ab 2 is 0.9637685198604251
f1 score of ab 3 is 0.9630934277708978
f1 score of ab 4 is 0.945610650786264
f1 score of ab 5 is 0.9746649815411104
f1 score of ab 6 is 0.9665417903333571
f1 score of ab 7 is 0.9573233412576465
f1 score of ab 8 is 0.9559245572013491
f1 score of ab 9 is 0.49876619370758796
 train data hamming_loss  : 0.0132
 train data precision recall f1  : (0.9482820512820513, 0.9612769230769231, 0.9496664224664224, None)


In [5]:
quarter_train = []
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values) 
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_attention_10net_fold2.csv").values) 
path = "./"
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold0.csv").values)
# quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_4block_10net_fold2.csv").values)

#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold0.csv").values)
#quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_one_net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold0.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold1.csv").values)
quarter_train.append(pd.read_csv(path+"quarter_final/"+"attention_10net_fold2.csv").values)

quarter_train.append(pd.read_csv(path+"quarter_final/"+"densenet_f0819_10net_fold.csv").values)

''' '''
# quarter_train_y = thr0*quarter_train[0] + thr1*quarter_train[1] + thr2*quarter_train[2] + \
#                 thr3*quarter_train[3] + thr4*quarter_train[4] + thr5*quarter_train[5]

# print("best_threshold :", best_threshold)

# quarter_y_pred = np.array([[1 if quarter_train_y[i, j] >= best_threshold[j] else 0 
#                             for j in range(quarter_train_y.shape[1])]
#           for i in range(len(quarter_train_y))])

# Restore
LR_clf = joblib.load("LR_ensemble.pkl")
MLkNN_clf = joblib.load("MLkNN10_ensemble.pkl")
MLkNN8_clf = joblib.load("MLkNN8_ensemble.pkl")

quarter_x_train = np.hstack(quarter_train)

quarter_y_pred_LR = LR_clf.predict(quarter_x_train)
quarter_y_pred_MLkNN = MLkNN_clf.predict(quarter_x_train).toarray()
quarter_y_pred_MLkNN8 = MLkNN8_clf.predict(quarter_x_train).toarray()

quarter_y_pred_MLkNN[:,1] = quarter_y_pred_MLkNN8[:,1]
quarter_y_pred_MLkNN[:,2] = quarter_y_pred_MLkNN8[:,2]

quarter_y_pred_MLkNN[:,3] = quarter_y_pred_LR[:,3]

quarter_y_pred_MLkNN[:,5] = quarter_y_pred_MLkNN8[:,5]
quarter_y_pred_MLkNN[:,6] = quarter_y_pred_MLkNN8[:,6]

quarter_y_pred_MLkNN[:,7] = quarter_y_pred_LR[:,7]

quarter_y_pred_MLkNN[:,9] = quarter_y_pred_MLkNN8[:,9]

quarter_y_pred = quarter_y_pred_MLkNN

print(" train data f1_score  :", f1_score(quarter_y_train, quarter_y_pred, average='macro'))

for i in range(10):
    print("f1 score of ab {} is {}".format(i, f1_score(quarter_y_train[:, i], quarter_y_pred[:, i], average='macro')))
    
print(" train data hamming_loss  :", hamming_loss(quarter_y_train, quarter_y_pred)) 
print(" train data precision recall f1  :", prf(quarter_y_train, quarter_y_pred,average="samples"))# 'micro', 'weighted'

 train data f1_score  : 0.8443423747565797
f1 score of ab 0 is 0.9723483730856107
f1 score of ab 1 is 0.9872106784931818
f1 score of ab 2 is 0.9637685198604251
f1 score of ab 3 is 0.9618120805912989
f1 score of ab 4 is 0.945610650786264
f1 score of ab 5 is 0.9746649815411104
f1 score of ab 6 is 0.9665417903333571
f1 score of ab 7 is 0.9525365906375869
f1 score of ab 8 is 0.9559245572013491
f1 score of ab 9 is 0.49876619370758796
 train data hamming_loss  : 0.013353846153846154
 train data precision recall f1  : (0.947, 0.9598410256410257, 0.9483638583638584, None)


In [ ]:
 train data f1_score  : 0.8442967714781059
f1 score of ab 0 is 0.9708833862292193
f1 score of ab 1 is 0.988758799327768
f1 score of ab 2 is 0.9629484127901154
f1 score of ab 3 is 0.9630934277708978
f1 score of ab 4 is 0.9440688933557289
f1 score of ab 5 is 0.9746300670156537
f1 score of ab 6 is 0.9673782455750232
f1 score of ab 7 is 0.9525056267020673
f1 score of ab 8 is 0.9555890425606381
f1 score of ab 9 is 0.49884348496530456
 train data hamming_loss  : 0.0134
 train data precision recall f1  : (0.9458333333333332, 0.9604820512820512, 0.9476832722832723, None)


In [ ]:
 train data f1_score  : 0.8445570284593916
f1 score of ab 0 is 0.972730581097661
f1 score of ab 1 is 0.988758799327768
f1 score of ab 2 is 0.9629484127901154
f1 score of ab 3 is 0.9630934277708978
f1 score of ab 4 is 0.9440688933557289
f1 score of ab 5 is 0.9746300670156537
f1 score of ab 6 is 0.9673782455750232
f1 score of ab 7 is 0.9525056267020673
f1 score of ab 8 is 0.9555890425606381
f1 score of ab 9 is 0.49884348496530456
 train data hamming_loss  : 0.013246153846153847
 train data precision recall f1  : (0.9482948717948717, 0.9615589743589743, 0.9496832722832723, None)


In [19]:
(0.844557 * 10 - 0.49884 +0.828)/10

0.877473

In [6]:
print(" train data f1_score  :", f1_score(quarter_y_train[:,:9], quarter_y_pred[:,:9], average='macro'))

 train data f1_score  : 0.9381581941739774
